In [6]:
import json
import numpy as np
import time
import pandas as pd
import spacy
import re
import string
import en_core_web_sm
import requests

/home/science/projects/simcube/venv/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [7]:
filelist=[]
root_path = '../../data/raw/covid/'

import os
for dirname, _, filenames in os.walk(root_path):
    for filename in filenames:
        filelist.append(os.path.join(dirname, filename))

In [8]:
triples=[]
nlp = en_core_web_sm.load()
w = "words.txt"
file = open(w,"r")
f = file.readlines()
word = []
for w in f:
    word.append(w.replace("\n", ""))

In [7]:
df_covid = pd.read_csv("../../data/processed/covid-2019-scientific-papers.csv")

In [8]:
df_covid.head()

,Unnamed: 0,paper_id,abstract,body_text,authors,title,journal,abstract_summary,abstract_word_count,body_word_count,lang
0,0,d604508a4de30a622bbe2d59d1f73abaaf9c14d2,NaN,"La pathologic virale du syst6me nerveux, nagub...","Cathala, F.",Généralités sur l'infection du système<br>ner...,Revue d'Electroencéphalographie et de Neurophy...,Not provided.,0,3899,fr
1,1,7cc9c36dd4de535aabba7346c12be7b470f6797f,Although promotion of safe hygiene is the sing...,Promotion of hygiene might be the single most ...,"Curtis, Val. Schmidt, Wolf...","Hygiene: new hopes, new horizons",The Lancet Infectious Diseases,Although promotion of safe hygiene is the<br>...,196,6480,en
2,2,b8069299efef2b8cf2d672b0174f6f0a7ef0580e,NaN,Oft ist die Unterscheidung zwischen infektiöse...,NaN,KAPITEL 13 Infektionskrankheiten,Innere Medizin,Not provided.,0,21808,de
3,3,53f7ce0a18de7792f478273f9449e175960563c0,Purpose: To explore factors relating to the pr...,Current global outbreak of severe acute respir...,"Wong, Chi-Yan. Tang, Catherine So-Kum",Practice of habitual and volitional health<br...,Journal of Adolescent Health,Purpose: To explore factors relating to the<b...,203,4004,en
4,4,e3c2df2221f21ddbb5ece5f094fa307e26b79e9e,Signal-dependent targeting of proteins into an...,"The mammalian cell is a highly organised, dyna...","Fulcher, Alex J.. Jans, David A.",Regulation of nucleocytoplasmic trafficking<b...,Biochimica et Biophysica Acta (BBA) - Molecula...,Signal-dependent targeting of proteins into<b...,196,8390,en


In [38]:
nlp = en_core_web_sm.load()
title = df_covid["title"][1]
title = nlp(title)

In [39]:
triples=[]
for token in title:
    if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
        if(len(token.text) > 2):
            triples.append(token.text)

In [40]:
triples

['Hygiene', 'hopes', 'horizons']

In [41]:
triples = list(set(triples) - set(word))

In [42]:
triples

['Hygiene', 'hopes', 'horizons']

In [52]:

nlp = en_core_web_sm.load()
text = df_covid["body_text"][1]
text = nlp(text)
triples=[]
for token in text:
    if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
        if(len(token.text) > 2):
            triples.append(token)


In [53]:
triples[:10]

[Promotion,
 hygiene,
 costeff,
 way,
 burden,
 disease,
 hygiene,
 subject,
 dollar,
 initiatives]

In [54]:
text = " ".join(triples)
text

TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [60]:
import string
from nltk.corpus import stopwords
pos = {'NOUN', 'PROPN', 'ADJ'}
stoplist = list(string.punctuation)
stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stoplist += stopwords.words('english')

extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=text, language='en')
extractor.candidate_selection(pos=pos, stoplist=stoplist)
extractor.candidate_weighting(threshold=0.74, method='average')
keyphrases = extractor.get_n_best(n=5)

ERROR:root:Cannot process <class 'spacy.tokens.doc.Doc'>


ValueError: The number of observations cannot be determined on an empty distance matrix.

In [63]:
nlp = en_core_web_sm.load()
text = df_covid["body_text"][1]
text = nlp(text)

# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=text,
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=10)

ERROR:root:Cannot process <class 'spacy.tokens.doc.Doc'>


In [64]:
keyphrases

[]

In [ ]:
literature = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    sha = str(row['sha'])
    if sha != 'nan':
        sha = sha + '.json';
        try:
            found = False
            with open(articles[sha]) as f:
                data = json.load(f)
                for key in ['abstract', 'body_text']:
                    if found == False and key in data:
                        for content in data[key]:
                            text = content['text']
                            if virus_match(text) == True:                                
                                literature.append({'file': articles[sha], 'body': text})                                
        except KeyError:
            pass

In [3]:
import spacy
import pytextrank
import en_core_web_sm

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."

# load a spaCy model, depending on language, scale, etc.
nlp = en_core_web_sm.load()

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)

0.1637     1  minimal generating sets
[minimal generating sets]
0.1440     4  systems
[systems, systems, systems, a system]
0.1200     3  solutions
[solutions, solutions, solutions]
0.1081     1  mixed types
[mixed types]
0.1080     1  strict inequations
[strict inequations]
0.1080     1  nonstrict inequations
[nonstrict inequations]
0.1063     1  diophantine equations
[Diophantine equations]
0.1044     1  a minimal supporting set
[a minimal supporting set]
0.1036     1  linear constraints
[linear constraints]
0.0954     1  a minimal set
[a minimal set]
0.0938     1  upper bounds
[Upper bounds]
0.0826     1  algorithms
[algorithms]
0.0824     1  natural numbers
[natural numbers]
0.0823     1  components
[components]
0.0814     1  diophantine
[Diophantine]
0.0810     1  all the considered types systems
[all the considered types systems]
0.0734     2  compatibility
[Compatibility, compatibility]
0.0716     1  construction
[construction]
0.0701     1  the set
[the set]
0.0651     2  crite

In [10]:

nlp = en_core_web_sm.load()
text = df_covid["body_text"][1]

import spacy
import pytextrank
import en_core_web_sm

# load a spaCy model, depending on language, scale, etc.
nlp = en_core_web_sm.load()

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)



0.0788    23  hygiene behaviour
[hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviours, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviours, hygiene behaviours, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour, hygiene behaviour]
0.0778     1  hygiene promotion programmes
[hygiene promotion programmes]
0.0761     8  hygiene practices
[hygiene practices, hygiene practice, hygiene practices, hygiene practices, hygiene practice, hygiene practices, hygiene practices, hygiene practices]
0.0755     1  hygiene behaviour change
[hygiene behaviour change]
0.0744     5  food hygiene
[food hygiene, food hygiene, food hygiene, food hygiene, Food hygiene]
0.0732     1  hygiene trials
[hygiene Trials]
0.0724     1  school hygiene
[school hygiene]
0.0723     1  

In [32]:
btext = df_covid["body_text"][0]
btext[:1000]

"La pathologic virale du syst6me nerveux, nagubre consid6r6e avec r6ticence, a acquis droit de cit6. Ce ne sont pas tellement la multiplicit6 des affections cliniques maintenant bien ~tudi6es, ni la diver-sit6 des virus << neurotropes >> qui font l'int6r6t de ce tr~s vaste domaine d'6tude, mais bien plus la r6v61ation de m6canismes physiopathologiques complexes et encore insuffisamment 6clair6s qui semblent avoir une r6elle port6e en pathologie g6n6rale. Ainsi les << infections virales lentes >~ (10) (31) qui nous offrent probablement des exemples de r6actions originales de l'organisme/a certaines modalit6s de l'infection.\nParmi beaucoup d'autres possibles, indiquons quelques-unes des questions que pose cette physiopathologic :\n--Quelles sont les voles de pb, n6tration des virus vers le syst6me nerveux ? et comment le virus s'y propage-t-il ? Y a-t-il des affinit6s particulibres entre certains virus et certains types cellulaires ? --Quels sont les effets de l'infection des cellules d

In [28]:
nlp = en_core_web_sm.load()

In [29]:
btext=nlp(btext)

In [33]:
type(btext)

str

In [34]:
btext[:1000]

"La pathologic virale du syst6me nerveux, nagubre consid6r6e avec r6ticence, a acquis droit de cit6. Ce ne sont pas tellement la multiplicit6 des affections cliniques maintenant bien ~tudi6es, ni la diver-sit6 des virus << neurotropes >> qui font l'int6r6t de ce tr~s vaste domaine d'6tude, mais bien plus la r6v61ation de m6canismes physiopathologiques complexes et encore insuffisamment 6clair6s qui semblent avoir une r6elle port6e en pathologie g6n6rale. Ainsi les << infections virales lentes >~ (10) (31) qui nous offrent probablement des exemples de r6actions originales de l'organisme/a certaines modalit6s de l'infection.\nParmi beaucoup d'autres possibles, indiquons quelques-unes des questions que pose cette physiopathologic :\n--Quelles sont les voles de pb, n6tration des virus vers le syst6me nerveux ? et comment le virus s'y propage-t-il ? Y a-t-il des affinit6s particulibres entre certains virus et certains types cellulaires ? --Quels sont les effets de l'infection des cellules d

In [31]:
extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=btext, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=5)

ERROR:root:Cannot process <class 'spacy.tokens.doc.Doc'>


ValueError: The number of observations cannot be determined on an empty distance matrix.

In [14]:
for file_name in filelist:
        with open(file_name,"r") as f:
            data = f.read()
        obj = json.loads(data)
        paperid = obj['paper_id']
        title = obj['metadata']['title']
        authors = obj['metadata']['authors']
        abstract = obj['abstract']
        bib = obj['bib_entries']
        body = obj['body_text']
        triples1=[]
        title = nlp(title)
        for token in title:
                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                        if(len(token.text) > 2):
                                triples1.append(token.text)
        triples1 = list(set(triples1) - set(word))
        for item in triples1:
                l=[item, paperid, 'has_title']
                triples.append(l)
        for item in authors:
                name = item['first'] + " " +  item['last']
                l = [name, paperid, 'has_author']
                triples.append(l)
        abss=""
        for item in abstract:
                abss = abss + item['text'] + " "
        abstract = re.sub(r" \d+", "", abss)
        abstract = re.sub(r"[^A-Za-z0-9 -]+", "",abstract)
        abstract = nlp(abstract)
        l1=[]
        abs1=abstract.ents
        for token in abstract:
                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                        l1.append(token.text)
        l2=[]
        for token in abs1:
                l2.append(token.text)
        l3 = list(set(l1) | set(l2))
        l3 = list(set(l3) - set(word))
        for item in l3:
                if(len(item) > 2):
                        l=[item, paperid, 'has_abstract']
                        triples.append(l)
        btext=""
        bod=[]
        for item in body:
                btext =item['text']
                keyphrases=[]
                try:
                        extractor = pke.unsupervised.TopicRank()
                        extractor.load_document(input=btext, language='en')
                        extractor.candidate_selection()
                        extractor.candidate_weighting()
                        keyphrases = extractor.get_n_best(n=5)
                except ValueError:
                        keyphrases=[]
                        btext = re.sub(r" \d+", "", btext)
                        btext = re.sub(r"[^A-Za-z0-9 -]+", "",btext)
                        btext=nlp(btext)
                        l1=[]
                        b1 = btext.ents
                        for token in btext:
                                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                                        l1.append(token.text)
                        l2=[]
                        for token in b1:
                                l2.append(token.text)
                        l3 = list(set(l1) | set(l2))
                        l3 = list(set(l3) - set(word))
                        for item2 in l3:
                                x=[item2]
                                keyphrases.append(x)
                for item in keyphrases:
                        print(item[0])
                        bod.append(item[0])
        bod = list(set(bod) - set(word))
        for item in bod:
                l=[item, paperid, 'has_body']
                triples.append(l)
        s=""
        for item in bib.keys():
                s = s + bib[item]['title'] + " "
        s = re.sub(r" \d+", "", s)
        s = re.sub("[^A-Za-z0-9 -]+", "",s)
        s = nlp(s)
        ref = s.ents
        l1=[]
        l2=[]
        for token in s:
                if((token.pos_ == "PROPN") or (token.pos_ == "NOUN")):
                        l1.append(token.text)
        for token in ref:
                l2.append(token.text)
        l3 = list(set(l1) | set(l2))
        l3 = list(set(l3) - set(word))
        for item in l3:
                if(len(item) > 2):
                        l=[item, paperid, 'has_reference']
                        triples.append(l)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
seen = set()
tripless=[]

count = 0
for item in triples:
        t = tuple(item)
        if t not in seen:
                tripless.append(item)
                seen.add(t)
triples=[]
for item in tripless:
        if (item[0].lower() in word):
                tripless.remove(item)
print(len(tripless))
for item in tripless:
        triples.append(item)
        entity = item[0]
        req = requests.get('https://id.nlm.nih.gov/mesh/lookup/term?label={query}&match=contains&limit=5'.format(query=entity.strip()),headers={"content-type": "application/json"}, verify=False)
        count=count+1
        print(count)
        if(count == 400):
                count = 0
                time.sleep(70)
        req1 = eval(req.text)
        for x in req1:
                rel = x['label']
                l = [rel, item[1], item[2]]
                triples.append(l)

triples = np.array(triples)
dy = pd.DataFrame(triples)
dy.to_csv("sampleresult1.csv", index=False)